In [1]:
import ragas

/mnt/docker/work/sd/notebooks/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
TOGETHER_API_KEY = os.env['TOGETHER_API_KEY']

In [18]:
import json
from ragas import SingleTurnSample, EvaluationDataset

with open("../dataset.json") as f:
    data = json.load(f)

questions = data['question']
ground_truths = data['ground_truths']
answers = data['answer']
contexts = data['contexts']

samples = []

for i in range(2):   # range(len(questions))
    sample = SingleTurnSample(
        user_input=questions[i], 
        retrieved_contexts=contexts[i], 
        response=answers[i],
        reference="\n".join(ground_truths[i])
    )
    samples.append(sample)

eval_dataset = EvaluationDataset(samples=samples)
eval_dataset

#from datasets import load_dataset
# eval_dataset = load_dataset("json", data_files="../dataset.json")
# eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=2)

In [58]:
from llama_index.embeddings.openai import OpenAIEmbedding
from ragas.embeddings import LlamaIndexEmbeddingsWrapper  # LangchainEmbeddingsWrapper

embedding_model_params = {
    "api_key": TOGETHER_API_KEY,
    "api_base": "https://api.together.xyz/v1",
    "model_name": "togethercomputer/m2-bert-80M-32k-retrieval"
}
embeddings = OpenAIEmbedding(**embedding_model_params)
evaluator_embeddings = LlamaIndexEmbeddingsWrapper(embeddings)
evaluator_embeddings

In [72]:
#evaluator_embeddings.embed_query("What's the most famous tower of Paris?")

In [73]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.metrics import ResponseRelevancy
from ragas.metrics import SemanticSimilarity
# from ragas.llms import LangchainLLMWrapper
from ragas.llms import LlamaIndexLLMWrapper
from ragas.metrics import TopicAdherenceScore
from llama_index.llms.together import TogetherLLM
from ragas import evaluate

llm_model_params = {
    "api_key": TOGETHER_API_KEY,
    "base_url": "https://api.together.xyz/v1",
    "model": "Qwen/Qwen2.5-7B-Instruct-Turbo"
}

chat_model = TogetherLLM(**llm_model_params)

evaluator_llm = LlamaIndexLLMWrapper(chat_model)
#rc = LLMContextRecall()

fc = FactualCorrectness()
ff = Faithfulness()
rr = ResponseRelevancy(embeddings = evaluator_embeddings)
ss = SemanticSimilarity(embeddings=evaluator_embeddings)

# needs multiturn samples dataset
# tas = TopicAdherenceScore(mode="precision", llm=evaluator_llm)

metrics = [fc, ff, rr, ss]
results = evaluate(dataset=eval_dataset, metrics=metrics, llm=evaluator_llm)

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:17<00:00,  2.14s/it]


In [74]:
df = results.to_pandas()
df.head()

,user_input,retrieved_contexts,response,reference,factual_correctness,faithfulness,answer_relevancy,semantic_similarity
0,What is ECMWF’s main focus in weather prediction?,[Seasonal Forecasting at ECMWF | CMEMS\n\r\nSk...,ECMWF's main focus in weather prediction is on...,ECMWF's main focus in weather prediction is on...,1.0,0.0,0.418660,0.692752
1,How does ECMWF use satellite data to improve w...,[Seasonal Forecasting at ECMWF | CMEMS\n\r\nSk...,ECMWF uses satellite data to improve weather f...,ECMWF uses satellite data to improve weather f...,1.0,0.0,0.936287,0.386332
